In [9]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_simulation_bayesimp as file

In [10]:
# Args setup
ntrial = 50
n = 500
n_int = 100
niter = 1000
front_door = True
minimise = False

In [11]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [12]:
# Submitting jobs
futures = []
for seed in range(ntrial):
        f = client.submit(file.main,
                              seed,
                              n,n_int,
                              niter = niter,
                              front_door = front_door,
                              minimise = minimise
                             )
        futures += [f]

In [17]:
futures

[<Future: cancelled, type: dict, key: main-c972094f208fd06b3cfb86c93cd45f1d>,
 <Future: cancelled, type: dict, key: main-96a58c8252a46ecc984333303b9bd361>,
 <Future: cancelled, type: dict, key: main-27a903df6dd8280a7f27d18976e17383>,
 <Future: cancelled, type: dict, key: main-470c8c24f909735ff72db482ea96d272>,
 <Future: cancelled, type: dict, key: main-2c4703b2d9ee3995a6e895e8bc00e288>,
 <Future: cancelled, type: dict, key: main-ebb15c6a381b2177908ffc8da42a90c3>,
 <Future: cancelled, type: dict, key: main-842eb5cf60ad138a996ca57f50d9f79a>,
 <Future: cancelled, type: dict, key: main-191a8e9b0eb2729ad9c2e74754e7b673>,
 <Future: cancelled, type: dict, key: main-8feebe3c892407f4d259bd265c37ad5e>,
 <Future: cancelled, type: dict, key: main-0bc3eeeeeed2d9cd1d159e5feffddb52>,
 <Future: cancelled, type: dict, key: main-5c4b88dd6066cb75d4f78a671b8cad59>,
 <Future: cancelled, type: dict, key: main-8440dfe31e306f2607975c26a059d41d>,
 <Future: cancelled, type: dict, key: main-d47de50d539957918756e

In [14]:
# Getting results
results = client.gather(futures)

In [15]:
# Closing client
client.close()
cluster.close()

In [16]:
# Saving results
torch.save(obj = results,
           f = "simulation_bayesimp_ntrial={0}_n={1}_frontdoor={2}_minimise={3}.pt".format(ntrial,
                                                                                           n,
                                                                                           front_door,
                                                                                           minimise)
          )